In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import rb_equivariant_cnn as conv
import rb_equivariant_gcnn as gconv

2024-08-01 14:12:06.049687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Setup

In [2]:
RB_CHANNELS = 4
HORIZONTAL_SIZE = 64
HEIGHT = 32

BATCH_SIZE = 64

# 3D Rayleigh-Bénard __Convolution__
- Equivariant to horizontal translations
- __No vertical parameter sharing__
- Height dependend bias
- Supports horizontal wrap (reflect) and same padding
    - Wrap makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This may destroy exact rotation equivariance in our experiments (nevertheless reflect will be preferable in practice)
- Also supports vertical same padding
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [3]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2,2,2), name='Conv1'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2,2,2), name='Conv2'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2,2,2), name='Conv3'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2,2,2), name='Conv4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (RB3D_Conv)               │ (64, 33, 33, 17, 4)    │        12,308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 17, 17, 9, 4)     │         6,516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (RB3D_Conv)               │ (64, 9, 9, 5, 4)       │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (RB3D_Conv)               │ (64, 5, 5, 3, 4)       │         2,172 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,616 (96.16 KB)

 Trainable params: 24,616 (96.16 KB)

 Non-trainable params: 0 (0.00 B)

# 3D Rayleigh-Bénard __D4 Group Equivariant Convolution__
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - __rotations around a vertical axis__
    - __reflections through a vertical plane__
    - __horizontal translations__
- __No vertical parameter sharing__
- Height dependend bias
- Supports horizontal wrap (reflect) padding
    - Makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This (as well as SAME padding) may destroy exact rotation equivariance in our experiments (nevertheless preferable in practice)
- Also supports vertical SAME padding
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [4]:
G = 'D4' # 'C4' for rotations or 'D4' for rotations and reflections
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'Lift_{G}_Conv1'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv2'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv3'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Lift_D4_Conv1 (RB3D_G_Conv)     │ (64, 33, 33, 8, 17, 4) │        12,308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv2 (RB3D_G_Conv)          │ (64, 17, 17, 8, 9, 4)  │        51,876 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv3 (RB3D_G_Conv)          │ (64, 9, 9, 8, 5, 4)    │        28,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv4 (RB3D_G_Conv)          │ (64, 5, 5, 8, 3, 4)    │        17,292 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,296 (430.84 KB)

 Trainable params: 110,296 (430.84 KB)

 Non-trainable params: 0 (0.00 B)

# Autoencoder

#### Convolutional

In [5]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), name='Conv1'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool1'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), name='Conv2'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool2'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), name='Conv3'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool3'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), name='Conv4'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (RB3D_Conv)               │ (64, 64, 64, 32, 4)    │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool1 (SpatialPooling)          │ (64, 32, 32, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 32, 32, 16, 4)    │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool2 (SpatialPooling)          │ (64, 16, 16, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (RB3D_Conv)               │ (64, 16, 16, 8, 4)     │         5,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool3 (SpatialPooling)          │ (64, 8, 8, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (RB3D_Conv)               │ (64, 8, 8, 4, 4)       │         2,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool4 (SpatialPooling)          │ (64, 4, 4, 2, 4)       │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,440 (169.69 KB)

 Trainable params: 43,440 (169.69 KB)

 Non-trainable params: 0 (0.00 B)

### D4 Group Equivariant Convolutional

In [6]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), 
                            name=f'Lift_{G}_Conv1'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool1'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1), 
                            name=f'{G}-Conv2'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool2'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1),
                            name=f'{G}-Conv3'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool3'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='REFLECT', v_padding='SAME', strides=(1,1,1),
                            name=f'{G}-Conv4'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

(64, 32, 32, 16, 8, 4)
(64, 16, 16, 8, 8, 4)
(64, 8, 8, 4, 8, 4)
(64, 4, 4, 2, 8, 4)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Lift_D4_Conv1 (RB3D_G_Conv)     │ (64, 64, 64, 8, 32, 4) │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool1 (SpatialPooling)   │ (64, 32, 32, 8, 16, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4-Conv2 (RB3D_G_Conv)          │ (64, 32, 32, 8, 16, 4) │        92,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool2 (SpatialPooling)   │ (64, 16, 16, 8, 8, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4-Conv3 (RB3D_G_Conv)          │ (64, 16, 16, 8, 8, 4)  │        46,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool3 (SpatialPooling)   │ (64, 8, 8, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4-Conv4 (RB3D_G_Conv)          │ (64, 8, 8, 8, 4, 4)    │        23,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool4 (SpatialPooling)   │ (64, 4, 4, 8, 2, 4)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,560 (720.94 KB)

 Trainable params: 184,560 (720.94 KB)

 Non-trainable params: 0 (0.00 B)